In [398]:
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
from glob import glob
import os
from pandas import DataFrame

In [399]:
def calc_cepstrum(data):
    """
    Calculates the complex cepstrum of a real sequence.
    """
    spectrum = np.fft.fft(data)
    log_spectrum = np.log(np.abs(spectrum))
    cepstrum = np.fft.ifft(log_spectrum).real
    return cepstrum


def find_fundamental_freq(samples, sr, freq_range):
    cepstrum = calc_cepstrum(samples)
    """
    Finding fundamental frequency of tested sound
    """

    min_freq, max_freq = freq_range
    start = int(sr / max_freq)
    end = int(sr / min_freq)
    narrowed_cepstrum = cepstrum[start:end]
    peak_ix = narrowed_cepstrum.argmax()
    freq0 = sr / (start + peak_ix)

    if freq0 < min_freq or freq0 > max_freq:
        return 0

    return freq0

In [400]:
audio_files = glob('Audio/piano single notes/*')

audio_names = []

for audio in audio_files:
    audio_names.append(audio[25:])
for count, file in enumerate(audio_names):
    # print(f"{count}. {file}")
    pass
audio_names2 = ['piano C2.wav', 'piano C#2.wav', 'piano D2.wav', 'piano D#2.wav', 'piano E2.wav', 'piano F2.wav',
                'piano F#2.wav', 'piano G2.wav', 'piano G#2.wav', 'piano A2.wav', 'piano A#2.wav', 'piano H2.wav',
                'piano C3.wav', 'piano C#3.wav', 'piano D3.wav', 'piano D#3.wav', 'piano E3.wav', 'piano F3.wav',
                'piano F#3.wav', 'piano G3.wav', 'piano G#3.wav', 'piano A3.wav', 'piano A#3.wav', 'piano H3.wav',
                'piano C4.wav', 'piano C#4.wav', 'piano D4.wav', 'piano D#4.wav', 'piano E4.wav', 'piano F4.wav',
                'piano F#4.wav', 'piano G4.wav', 'piano G#4.wav', 'piano A4.wav', 'piano A#4.wav', 'piano H4.wav',
                'piano C5.wav', 'piano C#5.wav', 'piano D5.wav', 'piano D#5.wav', 'piano E5.wav', 'piano F5.wav',
                'piano F#5.wav', 'piano G5.wav', 'piano G#5.wav', 'piano A5.wav', 'piano A#5.wav', 'piano H5.wav',
                'piano C6.wav', 'piano C#6.wav', 'piano D6.wav', 'piano D#6.wav', 'piano E6.wav', 'piano F6.wav',
                'piano F#6.wav', 'piano G6.wav', 'piano G#6.wav', 'piano A6.wav', 'piano A#6.wav', 'piano H6.wav',
                'piano C7.wav']
print(len(audio_names2))

61


In [401]:
SAMPLE_RATE = 44100
FREQUENCY_RANGE = (50, 2500)
WINDOW_SIZE = 8192
HOP_LENGTH = 1024

# For all files:

directory = os.getcwd()
audio_path = directory + '\Audio\piano single notes\\'
found_frequency = np.array([])

for audio_name in audio_names2:
    audio_path_actual = audio_path + audio_name
    y, sr = librosa.load(audio_path_actual,
                         sr=SAMPLE_RATE)  # y - raw data of audio file, sr - sample rate of audio file
    frames = librosa.util.frame(y, frame_length=WINDOW_SIZE, hop_length=HOP_LENGTH, axis=0)
    windowed_frames = np.hanning(WINDOW_SIZE) * frames
    f0_candidate = np.array([])

    for frame in windowed_frames:
        frequency = find_fundamental_freq(frame, sr=SAMPLE_RATE, freq_range=FREQUENCY_RANGE)
        f0_candidate = np.append(f0_candidate, frequency)
    f0 = np.median(f0_candidate)
    found_frequency =np.append(found_frequency, f0)
    print(f"{audio_name}, znaleziona f: {f0}")

piano C2.wav, znaleziona f: 66.11694152923538
piano C#2.wav, znaleziona f: 69.88906497622821
piano D2.wav, znaleziona f: 73.99328859060402
piano D#2.wav, znaleziona f: 78.46975088967972
piano E2.wav, znaleziona f: 83.05084745762711
piano F2.wav, znaleziona f: 88.2
piano F#2.wav, znaleziona f: 93.4322033898305
piano G2.wav, znaleziona f: 99.10112359550561
piano G#2.wav, znaleziona f: 104.75059382422803
piano A2.wav, znaleziona f: 111.36363636363636
piano A#2.wav, znaleziona f: 117.9144385026738
piano H2.wav, znaleziona f: 124.57627118644068
piano C3.wav, znaleziona f: 132.83132530120483
piano C#3.wav, znaleziona f: 140.0
piano D3.wav, znaleziona f: 148.4848484848485
piano D#3.wav, znaleziona f: 157.21975088967972
piano E3.wav, znaleziona f: 166.41509433962264
piano F3.wav, znaleziona f: 177.10843373493975
piano F#3.wav, znaleziona f: 187.6595744680851
piano G3.wav, znaleziona f: 199.54751131221718
piano G#3.wav, znaleziona f: 211.00478468899522
piano A3.wav, znaleziona f: 223.8578680203

In [402]:
notes = librosa.midi_to_note(np.arange(36, 97))
frequencies = librosa.midi_to_hz(np.arange(36, 97)).round(3)

df = DataFrame({'Note': notes, 'Frequency': frequencies, "Audio file": audio_names2})

df = DataFrame(
    {'Note': notes, 'Frequency [Hz]': frequencies, "Audio file": audio_names2, "Found frequency [Hz]": found_frequency.round(2)})
error = np.abs(df['Frequency [Hz]'] - df['Found frequency [Hz]'])
percentage_error = ((error/ frequencies)*100)
df['Error'] = error.round(2)
df['Error [%]'] = percentage_error.round(2)
status = []
for i in range(len(notes)):
    if i == len(frequencies) - 1:
        if error[i] > np.abs(frequencies[i] - frequencies[i - 1]):
            status.append('bad')
        else:
            status.append('good')
    else:
        if error[i] > np.abs(frequencies[i + 1] - frequencies[i]):
            status.append('bad')
        else:
            status.append('good')
df['Status'] = status
df.to_excel('test_f0.xlsx', sheet_name='sheet1', index=False)
df

,Note,Frequency [Hz],Audio file,Found frequency [Hz],Error,Error [%],Status
0,C2,65.406,piano C2.wav,66.12,0.71,1.09,good
1,C♯2,69.296,piano C#2.wav,69.89,0.59,0.86,good
2,D2,73.416,piano D2.wav,73.99,0.57,0.78,good
3,D♯2,77.782,piano D#2.wav,78.47,0.69,0.88,good
4,E2,82.407,piano E2.wav,83.05,0.64,0.78,good
...,...,...,...,...,...,...,...
56,G♯6,1661.219,piano G#6.wav,1633.33,27.89,1.68,good
57,A6,1760.000,piano A6.wav,1764.00,4.00,0.23,good
58,A♯6,1864.655,piano A#6.wav,1917.39,52.74,2.83,good
59,B6,1975.533,piano H6.wav,2004.55,29.02,1.47,good
